# Chapter 18: Reinforcement Learning

Reinforcement Learning (RL) merupakan cabang Machine Learning yang berfokus pada
bagaimana sebuah **agen** dapat belajar mengambil keputusan optimal
melalui interaksi langsung dengan sebuah **lingkungan (environment)**.

Berbeda dengan supervised learning yang bergantung pada data berlabel,
pada Reinforcement Learning agen belajar melalui proses *trial and error*.
Agen melakukan suatu **aksi (action)**, kemudian menerima **umpan balik (reward)**,
yang digunakan sebagai sinyal untuk memperbaiki perilaku di masa depan.

Tujuan utama Reinforcement Learning adalah mempelajari sebuah **kebijakan (policy)**
yang mampu memaksimalkan **total reward kumulatif** dalam jangka panjang,
bukan hanya keuntungan sesaat.


### Konsep Utama

Beberapa komponen fundamental dalam Reinforcement Learning meliputi:

1. **Agen dan Lingkungan**  
   Agen adalah entitas pembelajar, sedangkan lingkungan adalah dunia
   tempat agen berinteraksi dan bereaksi terhadap tindakan agen.

2. **State, Action, dan Reward**  
   - *State* merepresentasikan kondisi lingkungan saat ini  
   - *Action* adalah keputusan yang diambil agen  
   - *Reward* adalah umpan balik numerik yang menilai kualitas aksi

3. **Policy (Kebijakan)**  
   Policy adalah strategi yang menentukan aksi apa yang harus diambil
   oleh agen pada setiap state.

4. **OpenAI Gym**  
   Gym merupakan toolkit standar yang menyediakan berbagai environment
   simulasi untuk mengembangkan dan mengevaluasi algoritma RL.


## 1. Menyiapkan Lingkungan dengan OpenAI Gym

Sebagai contoh konkret, kita menggunakan environment klasik **CartPole**.
Tugas agen adalah menjaga sebuah tiang agar tetap berdiri tegak
dengan menggerakkan kereta ke kiri atau ke kanan.

Meskipun terlihat sederhana, CartPole merupakan benchmark penting
karena mencakup konsep keseimbangan, pengambilan keputusan cepat,
dan reward yang tertunda.


In [ ]:
import gym
import numpy as np

# Membuat environment CartPole
env = gym.make("CartPole-v1")

# Reset environment untuk mendapatkan state awal
obs = env.reset()
print("State awal:", obs)

# Contoh aksi acak
action = 1  # 0 = kiri, 1 = kanan
obs, reward, done, info = env.step(action)

print("State baru:", obs)
print("Reward:", reward)
print("Episode selesai:", done)


## 2. Neural Network Policies

Alih-alih menggunakan aturan statis (hard-coded),
kita dapat memanfaatkan **Neural Network** sebagai policy.
Model ini menerima state lingkungan sebagai input
dan menghasilkan probabilitas untuk setiap aksi.

Pendekatan ini memungkinkan agen untuk belajar
strategi yang jauh lebih fleksibel dan adaptif,
terutama pada lingkungan yang kompleks dan berdimensi tinggi.


In [ ]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 4  # Jumlah fitur state pada CartPole

policy_model = keras.models.Sequential([
    keras.layers.Dense(5, activation="relu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid")  # Probabilitas aksi ke kanan
])

print("Neural Network Policy berhasil dibuat.")


## 3. Algoritma Policy Gradients (REINFORCE)

Pendekatan **Policy Gradients** secara langsung mengoptimalkan policy,
bukan nilai Q atau fungsi nilai lainnya.

Algoritma REINFORCE bekerja dengan cara:
1. Menjalankan agen selama satu atau beberapa episode penuh
2. Menghitung total reward terdiskon untuk setiap langkah
3. Memperkuat aksi yang menghasilkan reward tinggi
   dan melemahkan aksi yang menghasilkan reward rendah

Pendekatan ini sangat cocok untuk:
- Action space kontinu
- Masalah dengan kebijakan stokastik


In [ ]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [
        discount_rewards(rewards, discount_factor)
        for rewards in all_rewards
    ]
    flat_rewards = np.concatenate(all_discounted_rewards)
    mean = flat_rewards.mean()
    std = flat_rewards.std()
    return [(rewards - mean) / std for rewards in all_discounted_rewards]

print("Fungsi reward processing siap digunakan.")


## 4. Q-Learning dan Markov Decision Process (MDP)

Sebagian besar masalah Reinforcement Learning dapat dimodelkan
sebagai **Markov Decision Process (MDP)**,
di mana state masa depan hanya bergantung pada state saat ini dan aksi yang diambil.

Q-Learning merupakan algoritma berbasis nilai (*value-based*)
yang bertujuan mempelajari fungsi Q(s, a),
yaitu estimasi kualitas suatu aksi pada state tertentu.


### Persamaan Bellman

Q(s, a) ← (1 − α) Q(s, a) + α [ r + γ max Q(s′, a′) ]

Di mana:
- α : learning rate
- γ : discount factor


## 5. Deep Q-Learning (DQN)

Ketika ruang state menjadi sangat besar atau kontinu,
pendekatan Q-table menjadi tidak praktis.
Solusinya adalah menggunakan **Deep Neural Network**
untuk mengaproksimasi fungsi Q.

Pendekatan ini dikenal sebagai **Deep Q-Network (DQN)**,
yang menjadi tonggak penting dalam kesuksesan RL modern,
terutama pada game Atari.


In [ ]:
from collections import deque

replay_buffer = deque(maxlen=2000)

def epsilon_greedy_policy(state, epsilon=0.1):
    if np.random.rand() < epsilon:
        return np.random.randint(2)
    else:
        Q_values = policy_model.predict(state[np.newaxis], verbose=0)
        return np.argmax(Q_values[0])

print("Komponen inti Deep Q-Learning siap.")


## Kesimpulan
1. Reinforcement Learning berfokus pada pengambilan keputusan jangka panjang
   dengan reward yang seringkali tertunda.
2. Policy Gradient cocok untuk masalah kompleks dan aksi kontinu.
3. Q-Learning dan DQN sangat efektif untuk lingkungan diskrit,
   namun membutuhkan teknik stabilisasi tambahan.
4. Untuk implementasi yang lebih matang,
   library seperti **TF-Agents** atau **Stable-Baselines**
   sangat direkomendasikan.
